In [1]:
from typing import Callable, Mapping, TypeVar, Sequence, Any, Iterable
from dataclasses import dataclass
from haskellian import either as E
from pipeteer import Pipeline, SimpleQueue, ReadQueue, Queue, Workflow, Wrapped, QueueKV, connect_queues, flatten_queues, input_queues
from example_workflow import Artifacts, Input, Output, workflow
from prettyprinter import pformat

In [2]:
Qin = QueueKV.sqlite(Input, 'queue.sqlite', 'Qin')
Qout = QueueKV.sqlite(Output, 'queue.sqlite', 'Qout')

def make_queue(path, type):
  return QueueKV.sqlite(type, 'queue.sqlite', '-'.join(path))

Qs = connect_queues(Qin, Qout, workflow, make_queue=make_queue)

In [3]:
Qins = input_queues(Qin, workflow, make_queue=make_queue)
flatten_queues(Qins)

[(('validation',),
  QueueKV(SQLiteKV(conn=<sqlite3.Connection object at 0x7ff4d047ae30>, table='validation', parse=<function SQLiteKV.validated.<locals>.<lambda> at 0x7ff4d04bf100>, dump=<function SQLiteKV.validated.<locals>.<lambda> at 0x7ff4d020cae0>, dtype='JSON'))),
 (('manual-correct',),
  QueueKV(SQLiteKV(conn=<sqlite3.Connection object at 0x7ff4d047b010>, table='manual-correct', parse=<function SQLiteKV.validated.<locals>.<lambda> at 0x7ff4d1394860>, dump=<function SQLiteKV.validated.<locals>.<lambda> at 0x7ff4d1394f40>, dtype='JSON')))]